# Imports

In [1]:
###########################################################################
##################### Import statements ###################################
###########################################################################

from scipy.stats import qmc                # Used for Latin Hypercube sampling
import pandas as pd                        # Pandas used for import/export
import numpy as np                         # Numpy for some math stuff
from scipy.integrate import odeint         # the odeint function is what we'll use to solve our system of ODEs
import matplotlib.pyplot as plt            # We use pyplot for plotting out our results 
import math                                # Used for some math operations
from scipy.interpolate import CubicSpline  # Used to create a spline from which compensation points can be calculated

# LHS Sampling

In [2]:
###########################################################################
##################### Latin Hypercube Sampling ############################
###########################################################################

Vc = (29/1000)
sampler = qmc.LatinHypercube(d=19)
sample = sampler.random(n=240000)


l_bounds = [0.25, 2, 1, 6.35, 4.7, 350, 0.5, 0.000000002, 0.00002248, 1, 26.6, 160000, 0.04, 129, 394.0, 18.5, Vc*0.5,1,1]
u_bounds = [2, 350, 7, 8, 7, 16000, 2.5, 0.0000012, 0.000185, 17.9, 98, 300000, 0.69, 238, 564.5, 31.3,Vc*1.5,2,2]

sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

sample_scaled_df = pd.DataFrame(sample_scaled)

sample_scaled_df.to_excel("Samples_240000_LowPerm_withQ10.xlsx")

# Parameter export

In [3]:
#########################################################################
################### Parameter Export ####################################
#########################################################################

# We define a function that can take the high-level parameters we vary for the LHS samples and calculate all 
# necessary parameters for the full model simulation

def parameter_calculation(CA_conc = 0.27, khcat = 0.3e6, KHC = 34, KCO2 = 1.5, pumpCost = 1, Vmax_BicA = 185e-6, PlipHCO3 = 2e-9, radCell = 1.5, q10_PlipHCO3 = 1, q10_PlipCO2 = 1, pH_cytosol = 8, pH_chloroplast = 7, Km_BicA = (0.217/1000)*1e6, generic_Q10=2,Nmem=1,temperature=45,PlipCO2=3.5e3,BicA_Vmax_factor=1,radBoundary=10,kin_Sco=238,Kc=29,Ko=480,Vc=(29/1000 * 3.730641276137879e-15),CO2m=8.469575316358753,O2m=191.42312950099154,HCm=0):
    
    # -- calculate cell size parameters  -- 
        
    radCell = radCell                                # um (the radius of the cell)
    radBoundary = radCell*2                          # um (the radius of the boundary)
    radCplast = radCell/2                            # um (the radius of the chloroplast)
    radPyr = radCell/4                               # um (the radius of the clear space in middle of chloroplast)
    SA_Pyr = 4*np.pi*(radPyr**2)                     # um^2 (the surface area of the clear space in middle of chloroplast)
    SA_Cplast = 4*np.pi*(radCplast**2)               # um^2 (the surface area of the chloroplast)
    SA_Cell = 4*np.pi*(radCell**2)                   # um^2 (the surface area of the cell)
    SA_Boundary = 4*np.pi*(radBoundary**2)           # um^2 (the surface area of the boundary)
    T1 = (4/3)*np.pi*(radPyr**3) / 1e15              # um^3, converted to L (stroma space volume)
    T2 = (4/3)*np.pi*(radCplast**3) / 1e15 - T1      # um^3, converted to L (chloroplast volume without the inner stroma space)
    T3 = (4/3)*np.pi*(radCell**3)  / 1e15 - T2       # um^3, converted to L (cytoplasm volume)
    T4 = (4/3)*np.pi*(radBoundary**3) / 1e15 - T3    # um^3, converted to L (boundary volume)
    LengthBoundary = radBoundary - radCell           # um (length of the boundary)
    
    # -- input temperature adjustment parameters --

    temperature = temperature # C
    q10_PlipCO2 = q10_PlipCO2
    q10_PlipHCO3 = q10_PlipHCO3
    q10_Ka_HCtransport = 2
    q10_Vc = 2.21 #vonCaemmerer book (Vcmax)
    q10_Kc = 2.24 #vonCaemmerer book
    q10_Ko = 1.63 #vonCaemmerer book
    q10_Kba = 2
    q10_k2 = 2 
    q10_k_2 = 2  
    q10_ShCO2_forward = 2
    q10_ShCO2_reverse = 2 
    q10_khcat = 2 
    q10_KCO2 = 2 
    q10_KHC = 2 
    q10_Vmax_BicA = 2 
    q10_Km_BicA = 2 
    q10_kin_Sco = 0.6 #Uemura paper

    # -- calculate conductivity of CO2 and bicarb through several concentric thylakoid layers -- 

    PlipCO2 = PlipCO2 * q10_PlipCO2**((temperature-25)/10)            # um/s (CO2 permeability coefficient of a double lipid layer membrane)
    PlipHCO3 = PlipHCO3 * (1e6) * q10_PlipHCO3**((temperature-25)/10) # um/s (bicarb permeability coefficient of a double lipid layer membrane)

    #generate list of surface areas of the stacked thylakoids (assume negligible thickness of the thylakoid membranes)

    thylakoid_areas = np.zeros(len(range(Nmem)))
    if Nmem != 1:
        thylakoid_additional_radius = (radCplast - radPyr) / (Nmem-1)
    else: 
        thylakoid_additional_radius = 0
    for i in range(Nmem):
        thylakoid_radius = radCplast - (i * thylakoid_additional_radius)
        thylakoid_areas[i] = 4*np.pi*(thylakoid_radius**2)
    
    #calculate total permeability of thylakoids TO CO2
    
    individual_resistance_thy_CO2 = np.zeros(len(thylakoid_areas))
    individual_resistance_total_CO2 = np.zeros(len(thylakoid_areas)+1)
    for i in range(len(thylakoid_areas)):
        individual_resistance_thy_CO2[i] = 1/(PlipCO2*thylakoid_areas[i])
        individual_resistance_total_CO2[i] = 1/(PlipCO2*thylakoid_areas[i])
        
    # Need to append one more value, representing the resistance of the plasmalemma, to the 2nd list
    
    individual_resistance_total_CO2[-1] = 1/(PlipCO2*SA_Cell)
    permeability_total_thy_CO2 = (sum(individual_resistance_thy_CO2))**(-1) #um^3/s
    permeability_total_cell_CO2 = (sum(individual_resistance_total_CO2))**(-1)

    #calculate total permeability of thylakoids TO HCO3-
    
    individual_resistance_thy_bicarb = np.zeros(len(thylakoid_areas))
    for i in range(len(thylakoid_areas)):
        individual_resistance_thy_bicarb[i] = 1/(PlipHCO3*thylakoid_areas[i])
    permeability_total_thy_bicarb = (sum(individual_resistance_thy_bicarb))**(-1) # um^3/s

    #adjust oxygen permeability as described in Fridlyand paper
    PthyO2 = np.sqrt(44/32)*permeability_total_cell_CO2 

    # -- set all other parameters and adjust for temperature effects -- 

    #calculated with PlipCO2 or PlipHCO3, which are temperature-adjusted above (do not need further temperature adjustment)
    
    Pmc = (PlipCO2*SA_Cell) / 1e15             # L/s (coefficient of CO2 permeability from medium to cytoplasm)
    Pcp = permeability_total_thy_CO2 / 1e15    # L/s (coefficient of CO2 permeability from stroma to cytoplasm)
    Po = PthyO2 / 1e15                         # L/s (coefficient of O2 permeability from thylakoid system to external medium)
    Pup = permeability_total_thy_bicarb / 1e15 # L/s (coefficient of bicarb permeability from chloroplast inner border to stroma)
    Puc = (PlipHCO3*SA_Cell) / 1e15            # L/s (coefficient of bicarb permeability from medium to cytoplasm)

    #other parameters with temperature adjustment already included or not relevant (do not need further temperature adjustment)
    CA_conc = CA_conc *1e6 / 1000              # mol m^-3, converted to uM, carbonic anhydrase concentration (using cytosolic value)
    pKa = 5.97                                 # CO2 to bicarb system overall pKa
    
    H_conc_cyt = 10**(-1*pH_cytosol)
    Keq_cyt = pH_cytosol - pKa 
    Keq_cyt = (10**Keq_cyt)*(H_conc_cyt)
    Keq_cyt = Keq_cyt * 1e6

    H_conc_cpl = 10**(-1*pH_chloroplast)
    Keq_cpl = pH_chloroplast - pKa
    Keq_cpl = (10**Keq_cpl)*(H_conc_cpl)
    Keq_cpl = Keq_cpl * 1e6

    protonc = H_conc_cyt*1e6  # uM
    protonp = H_conc_cpl*1e6  # uM 
    
    Henry_CO2 = 0.035                  # mol kg^-1 bar^-1 (standard-temperature Henry's law constant, CO2) (use this rather than 0.034 to be consistent with 1st paper)
    Henry_temp_CO2 = 2400              # K (Henry's law temperature dependence constant, CO2)
    Henry_O2 = 0.0013                  # mol kg^-1 bar^-1 (standard-temperature Henry's law constant, O2)
    Henry_temp_O2 = 1700               # K (Henry's law temperature dependence constant, O2)
    atm_pressure = 1.01325             # bar (atmospheric pressure)
    ppm_CO2 = 400/1000000              # ppm (the ppm of CO2 in the air)
    percent_O2 = 21                    # % (the % of O2 in the air)

    #parameters that do need temperature adjustment (inorganic carbon concentrations)

    Henry_adjusted_CO2 = Henry_CO2*np.exp(Henry_temp_CO2*((1/(temperature+273.15))-(1/298.15))) #mol kg^-1 bar^-1 (temperature-adjusted Henry's law constant, CO2)
    Henry_adjusted_O2 = Henry_O2*np.exp(Henry_temp_O2*((1/(temperature+273.15))-(1/298.15))) #mol kg^-1 bar^-1 (temperature-adjusted Henry's law constant, O2)
    CO2m_molkg = Henry_adjusted_CO2 * (atm_pressure*ppm_CO2) #mol kg^-1 (medium [CO2])
    O2m_molkg = Henry_adjusted_O2 * (atm_pressure*(percent_O2/100)) #mol kg^-1 (medium [O2])
    water_density = ((-5e-6*(temperature**2)) + (8e-6*temperature) + 1.0001)*10e2*0.001 #kg m^-3, converted to kg L^-1 (density of water at temperature)

    CO2m_calc = CO2m_molkg*water_density*1e6 # uM (medium [CO2])
    O2m_calc = O2m_molkg*water_density*1e6  # uM (medium [O2])
    HCm_calc = 0              # uM (medium [bicarbonate])

    #other parameters that do need temperature adjustment
    Vc = Vc*T1 * 4.5                                                      # mol/s (maximum rate of CO2 fixation inside the chloroplast); 4.8 derived from kcat ratio at 25 and 45 in Doug's data
    Kc = Kc                                                               # uM (apparent Michaelis constant for CO2 binding in the Calvin/CBB cycle)
    Ko = Ko                                                               # uM (apparent Michaelis constant for O2)
    k2 = 6.2e-2 * q10_k2**((temperature-25)/10)                           # s^-1
    k_2 = 1.37e1 * q10_k_2**((temperature-25)/10)                         # s^-1
    ShCO2_forward = 6e-2 * q10_ShCO2_forward**((temperature-25)/10)       # s^-1 # spontaneous hydration of CO2 (ShCO2)
    ShCO2_reverse = 2e1 * q10_ShCO2_reverse**((temperature-25)/10)        # s^-1
    khcat = khcat * q10_khcat**((temperature-25)/10)                      # s^-1, hydration rate constant
    KCO2 = KCO2 * 1e6 /1000 * q10_KCO2**((temperature-25)/10)             # mol m^-3, converted to uM, affinity of CA for CO2
    KHC = KHC * 1e6 /1000 * q10_KHC**((temperature-25)/10)                # mol m^-3, converted to uM, affinity of CA for bicarbonate
    Vmax_BicA = ((Vmax_BicA)/1e12) * q10_Vmax_BicA**((temperature-25)/10) # mol um^-2 s^-1, converted to...?
    Km_BicA = Km_BicA * q10_Km_BicA**((temperature-25)/10)                # umol L^-1
    kin_Sco = kin_Sco * q10_kin_Sco**((temperature-25)/10)                # a ratio (Uemura paper)
    
    DiffusionCoefficientO2 = 3.05e-5 * 1e8
    DiffusionCoefficientCO2 = (DiffusionCoefficientO2 * (np.sqrt(32/44))) #m^2 s

    Pmb = ((SA_Boundary * DiffusionCoefficientCO2) / (LengthBoundary)) / 1e15 # L/s (coefficient of CO2 permeability from external environment to boundary)
    Pmo = ((SA_Boundary * DiffusionCoefficientO2) / (LengthBoundary)) / 1e15  # L/s (coefficient of O2 permeability from external environment to boundary)
       
    return T1, T2, T3, Pmc, Pcp, Pup, Puc, Po, CO2m, O2m, HCm, CA_conc, Vc, Kc, Ko, k2, k_2, ShCO2_forward, ShCO2_reverse, khcat, KCO2, KHC, Vmax_BicA, Km_BicA, kin_Sco, Keq_cyt, Keq_cpl, Pmb, T4, pH_chloroplast, pH_cytosol, q10_PlipCO2, q10_PlipHCO3, PlipHCO3, pumpCost, PlipCO2, radCell, protonc, protonp, SA_Cplast, Pmo


In [15]:
#########################################################################
######## Converting LHS samples into final parameter sets ###############
#########################################################################

# We load in the LHS samples

input_dataframe = pd.read_excel("Samples_240000_LowPerm_withQ10.xlsx",index_col=0)

data = {
    'T1':[],
    'T2':[],
    'T3':[],
    'Pmc':[],
    'Pcp':[],
    'Pup':[],
    'Puc':[],
    'Po':[],
    'CO2m':[],
    'O2m':[],
    'HCm':[],
    'CA_conc':[],
    'Vc':[],
    'Kc':[],
    'Ko':[],
    'k2':[],
    'k_2':[],
    'ShCO2_forward':[],
    'ShCO2_reverse':[],
    'khcat':[],
    'KCO2':[],
    'KHC':[],
    'Vmax_BicA':[],
    'Km_BicA':[],
    'kin_Sco':[],
    'Keq_cyt':[], 
    'Keq_cpl':[],
    'Pmb':[],
    'T4':[],
    'pH_chloroplast':[],
    'pH_cytosol':[],
    'q10_Plip_CO2':[],
    'q10_Plip_HCO3':[],
    'PlipHCO3':[],
    'pumpCost':[],
    'PlipCO2':[],
    'radCell':[],
    'protonc':[],
    'protonp':[],
    'SA_Cplast':[],
    'Stacks':[],
    'Pmo':[]}

input_dataframe.columns = ["pumpCost", "Km_BicA", "Nmem", "pH_chloroplast", "pH_cytosol", "PlipCO2", "radCell", "PlipHCO3", "Vmax_BicA", "KCO2", "KHC", "khcat", "CA_conc", "kin_Sco", "Ko", "Kc", "Vc", "q10_PlipCO2", "q10_PlipHCO3"]

output_dataframe = pd.DataFrame(data)

for n in range(len(input_dataframe)):
    arguments = parameter_calculation(PlipCO2=input_dataframe['PlipCO2'][n],Nmem=int(np.round(input_dataframe['Nmem'][n],0)), Km_BicA = input_dataframe['Km_BicA'][n], 
        pH_chloroplast = input_dataframe['pH_chloroplast'][n], pH_cytosol = input_dataframe['pH_cytosol'][n], 
        q10_PlipCO2 = input_dataframe['q10_PlipCO2'][n], q10_PlipHCO3 = input_dataframe['q10_PlipHCO3'][n], 
        radCell = input_dataframe['radCell'][n], PlipHCO3 = input_dataframe['PlipHCO3'][n], 
        Vmax_BicA = input_dataframe['Vmax_BicA'][n], pumpCost = input_dataframe['pumpCost'][n], 
        KCO2 = input_dataframe['KCO2'][n], KHC = input_dataframe['KHC'][n], khcat = input_dataframe['khcat'][n],
        CA_conc = input_dataframe['CA_conc'][n], kin_Sco=input_dataframe['kin_Sco'][n], Ko = input_dataframe['Ko'][n], Kc = input_dataframe['Kc'][n],
                                     Vc = input_dataframe['Vc'][n])
                    
    output_dataframe.loc[n] = [arguments[0], arguments[1], arguments[2], arguments[3], arguments[4],
        arguments[5], arguments[6], arguments[7], arguments[8], arguments[9],
        arguments[10], arguments[11], arguments[12], arguments[13], arguments[14],
        arguments[15], arguments[16], arguments[17], arguments[18], arguments[19],
        arguments[20], arguments[21], arguments[22], arguments[23], arguments[24],
        arguments[25], arguments[26], arguments[27], arguments[28], arguments[29], 
        arguments[30], arguments[31], arguments[32], arguments [33], arguments[34], 
        arguments[35], arguments[36], arguments[37], arguments[38], arguments[39],input_dataframe['Nmem'][n],arguments[40]]

output_dataframe.to_csv()

dataframe_length = len(output_dataframe)

per_run = 400
n = 1

while n * per_run <= dataframe_length: 
    
    subset = output_dataframe.loc[n*per_run-per_run:n*(per_run)-1]
    subset.to_csv("Testparamssubset_{0}.csv".format(n))
    n = n + 1
                    
output_dataframe.to_csv("Latin_params.csv")

In [13]:
input_dataframe.columns = ["ATP_pump", "Km_BicA", "Nmem", "pH_chloroplast", "pH_cytosol", "PlipCO2", "radCell", "PlipHCO3", "Vmax_BicA", "KCO2", "KHC", "khcat", "CA_conc", "kin_Sco", "Ko", "Kc", "Vc", "q10_PlipCO2", "q10_PlipHCO3"]
input_dataframe.head(n=5)

,ATP_pump,Km_BicA,Nmem,pH_chloroplast,pH_cytosol,PlipCO2,radCell,PlipHCO3,Vmax_BicA,KCO2,KHC,khcat,CA_conc,kin_Sco,Ko,Kc,Vc,q10_PlipCO2,q10_PlipHCO3
0,1.786038,78.108434,5.989385,6.460856,6.004001,6870.920483,1.908827,1.199608e-06,0.000034,7.218772,28.703288,282019.643033,0.087829,225.870190,538.533512,23.356538,0.029980,1.031985,1.348831
1,1.580611,122.774135,6.639777,7.380412,5.707175,2376.002050,1.139468,5.796725e-07,0.000112,9.995838,27.353688,283985.434606,0.077095,150.591915,478.215003,25.543732,0.021718,1.711291,1.566950
2,0.958643,199.918403,1.254934,7.434408,5.391553,4957.089074,1.428497,6.332592e-07,0.000179,5.819245,81.028057,277504.633220,0.478262,187.471436,460.278334,25.906500,0.032807,1.563964,1.186358
3,0.496048,91.211790,5.133006,7.929893,6.613429,4870.127961,0.655126,2.054183e-08,0.000112,6.875820,44.928550,253026.540426,0.444295,200.149666,502.344562,25.430468,0.020550,1.516983,1.825959
4,1.572922,129.281389,4.809010,6.388476,6.517393,10430.677675,0.685937,9.759789e-07,0.000109,7.903170,42.754894,252884.837841,0.379739,227.098970,547.349588,21.785605,0.015026,1.394724,1.565728
